# Load Model

In [2]:
import os
import cv2
import numpy as np
from config import CWD, DATA_PATH, ANC_PATH, POS_PATH, NEG_PATH
import tensorflow as tf
from tensorflow.keras.layers import Layer

# Siamese L1 Distance class
class L1Dist(Layer):
    
    # Init method - inheritance
    def __init__(self, **kwargs):
        super().__init__()
       
    # Magic happens here - similarity calculation
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

# Reload model 
siamese_model = tf.keras.models.load_model('siamesemodelv2.h5', 
                                   custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy}

Face Verified: True, Confidence Score: 1.0
Face Verified: True, Confidence Score: 0.9999998807907104
Face Verified: True, Confidence Score: 0.9999996423721313
Face Verified: True, Confidence Score: 0.9999998807907104
Face Verified: True, Confidence Score: 0.9999998807907104
Face Verified: True, Confidence Score: 0.9999998807907104
Face Verified: True, Confidence Score: 1.0
Face Verified: True, Confidence Score: 0.9996569156646729
Face Verified: False, Confidence Score: 0.09416405856609344
Face Verified: False, Confidence Score: 0.050812557339668274


# Preprocess

In [1]:
def preprocess(file_path):
    
    # Read in image from file path
    byte_img = tf.io.read_file(file_path)
    # Load in the image 
    img = tf.io.decode_jpeg(byte_img)
    
    # Preprocessing steps - resizing the image to be 100x100x3
    img = tf.image.resize(img, (224,224))
    # Scale image to be between 0 and 1 
    img = img / 255.0

    # Return image
    return img

# Main function to run the face verification

In [ ]:
def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        # Make Predictions 
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
    
    # Detection Threshold: Metric above which a prediciton is considered positive 
    detection = np.sum(np.array(results) > detection_threshold)
    
    # Verification Threshold: Proportion of positive predictions / total positive samples 
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images'))) 
    verified = verification > verification_threshold
    
    return results, verified

if __name__ == "__main__":
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        frame = frame[120:120+250,200:200+250, :]

        cv2.imshow('Verification', frame)

        # Verification trigger
        if cv2.waitKey(10) & 0xFF == ord('v'):
            cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
            # Run verification
            results, verified = verify(siamese_model, 0.5, 0.5)
            print(f'Face Verified: {verified}, Confidence Score: {results[0][0][0]}')

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# References:
1. DeepFace: Closing the Gap to Human-Level Performance in Face Verification
2. Nicholas Renotte, https://lnkd.in/gi7fYrUY